# RAG FUSION
RAG FUSIONとは元の問い合わせから派生となるクエリを複数生成し、各クエリの検索結果をReciprocal Rank Fusionというアルゴリズムを用いてRe-ranking(順序付け)し、関連度の高いものを抽出する手法です

##  下準備

In [20]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter,RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Document load
loader = DirectoryLoader("../datasets/company_documents_dataset_1/", glob="**/*.txt",recursive=True)
raw_docs = loader.load()

# Document split
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    return_each_line=False,
    strip_headers = False 
)
docs = []
for raw_doc in raw_docs:
    source = raw_doc.metadata["source"]
    spilited_docs = markdown_splitter.split_text(raw_doc.page_content)
    for doc in spilited_docs:
        doc.metadata["source"] = source#metadataにsourceを加える
    docs = docs + spilited_docs
markdown_splited_docs = docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 800,chunk_overlap=50)
docs = text_splitter.split_documents(docs)

# Embd
vectorstore = Chroma.from_documents(persist_directory="./vecstore/index", documents=docs, embedding=OpenAIEmbeddings())

#llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)





## 複数のqueryを作成するpromptの実装

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler

template = """AI言語モデルアシスタントとして、与えられたユーザーの質問から関連するドキュメントをベクトルデータベースから検索するために、その質問の異なる5つのバージョンを生成することがあなたの任務です。質問に対する複数の視点を生成することで、距離ベースの類似性検索のいくつかの制限を克服するのをユーザーに助けます。
これらの代替質問を改行で区切って提供してください。オリジナルの質問: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))#改行コードごとにsplitで区切ってリストにする
)

#動作確認
question = "社長の略歴を教えて"
handler = ConsoleCallbackHandler()
result = generate_queries.invoke({"question":question},{"callbacks":[handler]})
print("結果:",result)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "社長の略歴を教えて"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "社長の略歴を教えて"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "AI言語モデルアシスタントとして、与えられたユーザーの質問から関連するドキュメントをベクトルデータベースから検索するために、その質問の異なる5つのバージョンを生成することがあなたの任務です。質問に対する複数の視点を生成することで、距離ベースの類似性検索のいくつかの制限を克服するのをユーザーに助けます。\nこれらの代替質問を改行で区切って提供してください。オリジナルの質問: 社長の略歴を教えて",
          "additional_kwargs": {}
        }
      }
    ]
  }
}
[llm/start] [1:chain:RunnableSe

## Reciprocal Rank Fusion関数の実装とretreaval chainの作成

参考
https://qiita.com/isanakamishiro2/items/f4387443b86723eecf36

In [18]:
from langchain.load import dumps, loads
def reciprocal_rank_fusion(results: list[list], k=10):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results


# retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

question = "残業手当はいくら？"
retrieval_chain = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain.invoke({"question": question},{"callbacks":[handler]})
print("-"*150)
for doc in docs:
    print(doc)
    print("")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "残業手当はいくら？"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "残業手当はいくら？"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "AI言語モデルアシスタントとして、与えられたユーザーの質問から関連するドキュメントをベクトルデータベースから検索するために、その質問の異なる5つのバージョンを生成することがあなたの任務です。質問に対する複数の視点を生成することで、距離ベースの類似性検索のいくつかの制限を克服するのをユーザーに助けます。\nこれらの代替質問を改行で区切って提供してください。オリジナルの質問: 残業手当はいくら？",
          "additional_kwargs": {}
        }
      }
    ]
  }
}
[llm/start] [1:chain:RunnableSe

# Chain

In [19]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# RAG
rag_template = """Answer the following question based on this context:

{context}

Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | rag_prompt
    | llm
    | StrOutputParser()
)
handler = ConsoleCallbackHandler()
rag_chain.invoke({"question":question},{"callbacks":[handler]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "残業手当はいくら？"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "question": "残業手当はいくら？"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 3:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "残業手当はいくら？"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 3:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "残業手当はいくら？"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "残業手当はいくら？"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "残業手当はいくら？"
}
[chain/end] [1:chain:Runnable

[llm/end] [1:chain:RunnableSequence > 17:llm:ChatOpenAI] [1.89s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "残業手当は、基本給／１か月の平均所定労働時間数×１．２５×時間外労働時間数により支給されます。",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "残業手当は、基本給／１か月の平均所定労働時間数×１．２５×時間外労働時間数により支給されます。",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 51,
      "prompt_tokens": 1775,
      "total_tokens": 1826
    },
    "model_name": "gpt-3.5-turbo",
    "system_fingerprint": "fp_69829325d0"
  },
  "run": null
}
[chain/start] [1:chain:RunnableSequence > 18:parser:StrOutputPars

'残業手当は、基本給／１か月の平均所定労働時間数×１．２５×時間外労働時間数により支給されます。'

# reference
https://arxiv.org/abs/2402.03367

https://qiita.com/isanakamishiro2/items/f4387443b86723eecf36

https://github.com/Raudaschl/rag-fusion/blob/master/main.py
